In [5]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import os

key = os.environ.get('OPENAI_API_KEY')
openai.api_key = key

GPT_MODEL = "gpt-3.5-turbo-0613"

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

In [20]:
functions = [
    {
        "name": "search_items",
        "description": "Search for items in the inventory database",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "A list of item names",
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "add_or_update_item",
        "description": "Add a new item to the inventory or update it if it already exists",
        "parameters": {
            "type": "object",
            "properties": {
                "item_name": {
                    "type": "string",
                    "description": "The name of the item to add or update",
                },
                "additional_quantity": {
                    "type": "integer",
                    "description": "The additional quantity to add to the inventory (defaults to 1)",
                    "default": 1
                }
            },
            "required": ["item_name"]
        }
    },
    {
        "name": "delete_items",
        "description": "Delete items from the inventory database",
        "parameters": {
            "type": "object",
            "properties": {
                "items": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "A list of item names to delete from the inventory",
                }
            },
            "required": ["items"]
        }
    }
]


In [6]:
messages = []
messages.append({
    "role": "system", 
     "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "I would like to add something"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant', 'content': 'Sure! What would you like to add?'}

In [7]:
messages.append({"role": "user", "content": "A few green LEDs"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'add_or_update_item',
  'arguments': '{\n  "item_name": "green LED",\n  "additional_quantity": 3\n}'}}

In [9]:
chat_response.json()

{'id': 'chatcmpl-8HLTZWTNbjlOWbQJzyLNxrCGputwf',
 'object': 'chat.completion',
 'created': 1699144821,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'add_or_update_item',
     'arguments': '{\n  "item_name": "green LED",\n  "additional_quantity": 3\n}'}},
   'finish_reason': 'function_call'}],
 'usage': {'prompt_tokens': 211, 'completion_tokens': 27, 'total_tokens': 238}}

In [12]:
messages.append({"role": "user", "content": "Where are my leds?"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'search_items',
  'arguments': '{\n  "query": "LED"\n}'}}

In [21]:
messages.append({"role": "user", "content": "Where are my leds and batteries?"})
chat_response = chat_completion_request(
    messages, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'search_items',
  'arguments': '{\n  "query": ["LEDs", "batteries"]\n}'}}

In [19]:
chat_response.json()

{'error': {'message': "Invalid schema for function 'search_items': 'list' is not valid under any of the given schemas",
  'type': 'invalid_request_error',
  'param': None,
  'code': None}}

In [ ]:
def execute_function_call(message):
    if message["function_call"]["name"] == "search_items":
        query = json.loads(message["function_call"]["arguments"])["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

In [ ]:
if assistant_message.get("function_call"):
    results = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})
pretty_print_conversation(messages)